<a href="https://colab.research.google.com/github/ranimsaidi/image-processing-ML/blob/main/Copie_de_apr%C3%A9s_r%C3%A9haussement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def lbp_extractor(pic):
  from skimage import feature 
  import numpy as np
  import cv2
  
  

  def lbp_features(img):
    
    # LBP operates in images of only one channel, here we go converter
    
    # RGB for cinza scaling using the Luminance method
     #= img[:,:,0].astype(float)*0.3 + img[:,:,1].astype(float)*0.59 + img[:,:,2].astype(float)*0.11
    img_gray= cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2GRAY)
    
    
    # here we define the number of points and the radius, padrao = 8, 1
    lbp = feature.local_binary_pattern(img_gray.astype(np.uint8), 3, 3, method="uniform")
    
    # lbp returns a matrix with the codes, so we must extract the histogram
    (hist, _) = np.histogram(lbp, bins=2**12)

    # normalize or histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    # return the histogram of Local Binary Patterns

    return hist
  img1 = "ref_verylow.png"
  img2 = "ref_low.png"
  img3 = "ref_medium.png"
  img4 = "ref_high.png"  
  lbp1 = lbp_features(img1)
  lbp2 = lbp_features(img2)
  lbp3 = lbp_features(img3)
  lbp4 = lbp_features(img4) 
  
  lbp = lbp_features(pic)
  score1=cv2.compareHist(lbp.astype(np.float32),lbp1.astype(np.float32), cv2.HISTCMP_CORREL)  
  score2=cv2.compareHist(lbp.astype(np.float32),lbp2.astype(np.float32),  cv2.HISTCMP_CORREL)       
  score3=cv2.compareHist(lbp.astype(np.float32),lbp3.astype(np.float32),  cv2.HISTCMP_CORREL)  
  score4=cv2.compareHist(lbp.astype(np.float32),lbp4.astype(np.float32),  cv2.HISTCMP_CORREL) 
  return score1,score2,score3,score4

In [ ]:
def hog_extractor(img):

 import cv2
 import numpy as np
 def hog_features(pic):
    import matplotlib.pyplot as plt
    from skimage import io
    from skimage import color
    from skimage.transform import resize
    import math
    from skimage.feature import hog
    import numpy as np
    
    img = resize(color.rgb2gray(io.imread(pic)), (128, 64))
    
    
    img = np.array(img)
    
    
    mag = []
    theta = []
    for i in range(128):
      magnitudeArray = []
      angleArray = []
      for j in range(64):
        # Condition for axis 0
        if j-1 <= 0 or j+1 >= 64:
          if j-1 <= 0:
            # Condition if first element
            Gx = img[i][j+1] - 0
          elif j + 1 >= len(img[0]):
            Gx = 0 - img[i][j-1]
        # Condition for first element
        else:
          Gx = img[i][j+1] - img[i][j-1]
        
        # Condition for axis 1
        if i-1 <= 0 or i+1 >= 128:
          if i-1 <= 0:
            Gy = 0 - img[i+1][j]
          elif i +1 >= 128:
            Gy = img[i-1][j] - 0
        else:
          Gy = img[i-1][j] - img[i+1][j]
    
        # Calculating magnitude
        magnitude = math.sqrt(pow(Gx, 2) + pow(Gy, 2))
        magnitudeArray.append(round(magnitude, 9))
    
        # Calculating angle
        if Gx == 0:
          angle = math.degrees(0.0)
        else:
          angle = math.degrees(abs(math.atan(Gy / Gx)))
        angleArray.append(round(angle, 9))
      mag.append(magnitudeArray)
      theta.append(angleArray)
    
    mag = np.array(mag)
    
    
    theta = np.array(theta)
    
    number_of_bins = 9
    step_size = 180 / number_of_bins
    
    def calculate_j(angle):
      temp = (angle / step_size) - 0.5
      j = math.floor(temp)
      return j
    
    def calculate_Cj(j):
      Cj = step_size * (j + 0.5)
      return round(Cj, 9)
    
    def calculate_value_j(magnitude, angle, j):
      Cj = calculate_Cj(j+1)
      Vj = magnitude * ((Cj - angle) / step_size)
      return round(Vj, 9)
    
    histogram_points_nine = []
    for i in range(0, 128, 8):
      temp = []
      for j in range(0, 64, 8):
        magnitude_values = [[mag[i][x] for x in range(j, j+8)] for i in range(i,i+8)]
        angle_values = [[theta[i][x] for x in range(j, j+8)] for i in range(i, i+8)]
        for k in range(len(magnitude_values)):
          for l in range(len(magnitude_values[0])):
            bins = [0.0 for _ in range(number_of_bins)]
            value_j = calculate_j(angle_values[k][l])
            Vj = calculate_value_j(magnitude_values[k][l], angle_values[k][l], value_j)
            Vj_1 = magnitude_values[k][l] - Vj
            bins[value_j]+=Vj
            bins[value_j+1]+=Vj_1
            bins = [round(x, 9) for x in bins]
        temp.append(bins)
      histogram_points_nine.append(temp)
    
    epsilon = 1e-05
    
    feature_vectors = []
    for i in range(0, len(histogram_points_nine) - 1, 1):
      temp = []
      for j in range(0, len(histogram_points_nine[0]) - 1, 1):
        values = [[histogram_points_nine[i][x] for x in range(j, j+2)] for i in range(i, i+2)]
        final_vector = []
        for k in values:
          for l in k:
            for m in l:
              final_vector.append(m)
        k = round(math.sqrt(sum([pow(x, 2) for x in final_vector])), 9)
        final_vector = [round(x/(k + epsilon), 9) for x in final_vector]
        temp.append(final_vector)
      feature_vectors.append(temp)
    
    
    hogg_features = np.array(feature_vectors)
    
    hog_reshaped=hogg_features.reshape(3780,)
    return hog_reshaped
 img1 = "ref_verylow.png"
 img2 = "ref_low.png"
 img3 = "ref_medium.png"
 img4 = "ref_high.png"  
 hog1 = hog_features(img1)
 hog2 = hog_features(img2)
 hog3 = hog_features(img3)
 hog4 =hog_features(img4) 
 hogg = hog_features(img)
 score1=cv2.compareHist(hogg.astype(np.float32),hog1.astype(np.float32), cv2.HISTCMP_CORREL)  
 score2=cv2.compareHist(hogg.astype(np.float32),hog2.astype(np.float32),  cv2.HISTCMP_CORREL)       
 score3=cv2.compareHist(hogg.astype(np.float32),hog3.astype(np.float32),  cv2.HISTCMP_CORREL)  
 score4=cv2.compareHist(hogg.astype(np.float32),hog4.astype(np.float32),  cv2.HISTCMP_CORREL) 
 return score1,score2,score3,score4

In [ ]:
!pip install mahotas

In [ ]:
def haralick_extractor(pic):  
  # importing required libraries
  import numpy as np
  import mahotas
  from pylab import imshow, show

  # loading image
  img = mahotas.imread(pic)
    
  # filtering the image
  img = img[:, :, 0]
    
  # setting gaussian filter
  gaussian = mahotas.gaussian_filter(img, 15)

  # setting threshold value
  gaussian = (gaussian > gaussian.mean())

  # making is labelled image
  labeled, n = mahotas.label(gaussian)


  

  # getting haralick features
  h_feature = mahotas.features.haralick(labeled)
  liste= [*h_feature[0], *h_feature[1], *h_feature[2],*h_feature[3]] 
  return liste


In [ ]:
def mean_std_bgr(image):
  import cv2
  img = cv2.imread(image)
  Mean = []
  standardDev = []
  color = ('b','g','r')
  for i,col in enumerate(color):
    mean, std = cv2.meanStdDev(img[:,:,i])
    Mean.append(mean[0][0])
    standardDev.append(std[0][0])
  return  Mean[0], Mean[1], Mean[2], standardDev[0], standardDev[1], standardDev[2]

#finished

In [ ]:
#Moyenne de chaque canal et Standard Deviation (HSV)
def mean_std_hsv(img):
  import cv2
  hsv_img = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2HSV)
  Mean = []
  standardDev = []
  channels = ('h','s','v')
  for i,channel in enumerate(channels):
    mean, std = cv2.meanStdDev(hsv_img[:,:,i])
    Mean.append(mean[0][0])
    standardDev.append(std[0][0])
  return  Mean[0], Mean[1], Mean[2], standardDev[0], standardDev[1], standardDev[2]

#finished

In [ ]:
#Histogramme des canaux BGR par image
def hist_bgr(img):
  import cv2
  import numpy as np
  img=cv2.imread(img)
  img1 = cv2.imread("ref_verylow.png")
  img2 = cv2.imread("ref_low.png")
  img3 = cv2.imread("ref_medium.png")
  img4 = cv2.imread("ref_high.png")

  hist = []
  color = ('b','g','r')
  for i,col in enumerate(color):
      histr = cv2.calcHist([img],[i],None,[256],[0,256])
      histr1 = cv2.calcHist([img1],[i],None,[256],[0,256])
      histr2 = cv2.calcHist([img2],[i],None,[256],[0,256])
      histr3 = cv2.calcHist([img3],[i],None,[256],[0,256])
      histr4 = cv2.calcHist([img4],[i],None,[256],[0,256])

  score1=cv2.compareHist(histr.astype(np.float32),histr1.astype(np.float32), cv2.HISTCMP_CORREL )  
  score2=cv2.compareHist(histr.astype(np.float32),histr2.astype(np.float32),  cv2.HISTCMP_CORREL)       
  score3=cv2.compareHist(histr.astype(np.float32),histr3.astype(np.float32),  cv2.HISTCMP_CORREL)  
  score4=cv2.compareHist(histr.astype(np.float32),histr4.astype(np.float32),  cv2.HISTCMP_CORREL)

  return score1, score2, score3, score4

 #finished

In [ ]:
#Histogramme des canaux HSV par image
def hist_hsv(img):
  import cv2
  import numpy as np

  img1 = cv2.cvtColor(cv2.imread("ref_verylow.png"), cv2.COLOR_BGR2HSV)
  img2 = cv2.cvtColor(cv2.imread("ref_low.png"), cv2.COLOR_BGR2HSV)
  img3 = cv2.cvtColor(cv2.imread("ref_medium.png"), cv2.COLOR_BGR2HSV)
  img4 = cv2.cvtColor(cv2.imread("ref_high.png"), cv2.COLOR_BGR2HSV)

  hsv_img = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2HSV)
  hist = []
  channel = ('hc','sm','vy')
  for i,ch in enumerate(channel):
      histr = cv2.calcHist([hsv_img],[i],None,[256],[0,256])
      histr1 = cv2.calcHist([img1],[i],None,[256],[0,256])
      histr2 = cv2.calcHist([img2],[i],None,[256],[0,256])
      histr3 = cv2.calcHist([img3],[i],None,[256],[0,256])
      histr4 = cv2.calcHist([img4],[i],None,[256],[0,256])

  score1=cv2.compareHist(histr.astype(np.float32),histr1.astype(np.float32), cv2.HISTCMP_CORREL)  
  score2=cv2.compareHist(histr.astype(np.float32),histr2.astype(np.float32),  cv2.HISTCMP_CORREL)       
  score3=cv2.compareHist(histr.astype(np.float32),histr3.astype(np.float32),  cv2.HISTCMP_CORREL)  
  score4=cv2.compareHist(histr.astype(np.float32),histr4.astype(np.float32),  cv2.HISTCMP_CORREL)

  return score1, score2, score3, score4

 #finished

In [ ]:
def bgr_distribution(img):
  import cv2
  import numpy as np
  dist = []
  # Extract each colour channel
  img=cv2.imread(img)
  blue, green, red = img[:,:,0], img[:,:,1], img[:,:,2]

  # Total red+green+blue intensity
  intensity = img.sum(axis=2)

  # Function to calculate proportion of a certain channel
  def colour_frac(color):
      return np.sum(color)/np.sum(intensity)

  # Calculate the proportion of each colour
  blue_fraction = colour_frac(blue)
  green_fraction = colour_frac(green)
  red_fraction = colour_frac(red)
  
  dist.append(blue_fraction)
  dist.append(green_fraction)
  dist.append(red_fraction)
  #sum_colour_fraction = red_fraction + green_fraction + blue_fraction
  #print('Red fraction: {}'.format(red_fraction))
  #print('\nGreen fraction: {}'.format(green_fraction))
  #print('\nBlue fraction: {}'.format(blue_fraction))
  return blue_fraction, green_fraction, red_fraction

#finishied

In [ ]:
def image_colorfulness(image):
  import cv2
  import numpy as np
  #rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	# split the image into its respective RGB components
  (B, G, R) = cv2.split(cv2.imread(image).astype("float"))
	# compute rg = R - G
  rg = np.absolute(R - G)
	# compute yb = 0.5 * (R + G) - B
  yb = np.absolute(0.5 * (R + G) - B)
	# compute the mean and standard deviation of both `rg` and `yb`
  (rbMean, rbStd) = (np.mean(rg), np.std(rg))
  (ybMean, ybStd) = (np.mean(yb), np.std(yb))
	# combine the mean and standard deviations
  stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
  meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
	# derive the "colorfulness" metric and return it
	#return stdRoot + (0.3 * meanRoot)
  return   stdRoot + (0.3 * meanRoot)



#finished

In [ ]:
def contrast(img):
  import cv2
  img=cv2.imread(img)
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  contrast = gray_img.std()
  return contrast



#finished

In [ ]:
#brightness
def brightness(img):
   from PIL import Image, ImageStat
   im = Image.open(img).convert('L')
   stat = ImageStat.Stat(im)
   return stat.mean[0]


In [ ]:
#sharpness
def sharpness(img): 
  import cv2
  import numpy as np
  import matplotlib.pyplot as plt
  import math
  img=cv2.imread(img)

  def divisors(n):
    result = set()
    for i in range(1, int(n**0.5)+1):
        if n % i == 0:
            result.add(i)
            result.add(n//i)
    return list(result)

  def reshape_split(image: np.ndarray, kernel_size: tuple):
    img_height, img_width = image.shape
    tile_height, tile_width = kernel_size
    tiled_array = image.reshape(img_height//tile_height, tile_height, img_width//tile_width, tile_width)
    tiled_array = tiled_array.swapaxes(1,2)
    return tiled_array

  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_blur = cv2.GaussianBlur(gray_img, (3,3), 0)

  sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5)
  #plt.imshow(sobelxy)
  #print(img.shape)
  k1 = divisors(sobelxy.shape[0])[1]
  k2 = divisors(sobelxy.shape[1])[1]
  l1, l2 =(int(sobelxy.shape[0]/k1),int(sobelxy.shape[1]/k2)) 
  #gives 2x2 images
  tiles = reshape_split(gray_img, (l1, l2))

  #print(tiles)
  max_images = []
  min_images = []
  EME = 0
  for i in range(tiles.shape[0]): #go through rows of pictures (in case of 4 pictures: 2 rows)
    for j in range(tiles.shape[1]):#go through columns of pictures (in case of 4 pictures: 2 columns)
      max_rows = []
      min_rows = []
      for row in range(tiles.shape[2]):
        max_rows.append(max(tiles[i][j][row]))
        min_rows.append(min((tiles[i][j][row])))
      max_images.append(max(max_rows))
      min_images.append(min(min_rows))

  #print(max_images, min_images)
  k1 = tiles.shape[0]
  k2 = tiles.shape[1]

  for i in range(k1*k2):
    
      if max_images[i] == 0:
        a = 1 
      else: 
        a = max_images[i]
      if min_images[i] == 0:
        b = 1 
      else: 
        b = min_images[i]
      EME = EME + math.log(a/b)
      EME = (EME*2)/k1*k2

  return EME

sharpness("ref_high.png")

#finished

putting it all in a dataframe

In [ ]:
#connexion au google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
dataset_path="/content/drive/MyDrive/denoised/"
path_very_low="/content/drive/MyDrive/denoised/very_Low_Turbidity/"
path_low="/content/drive/MyDrive/denoised/Low_Turbidity/"

path_med="/content/drive/MyDrive/denoised/Medium_Turbidity/"
path_high="/content/drive/MyDrive/denoised/High_Turbidity/"

very_low_dir=os.path.join(path_very_low)
low_dir = os.path.join(path_low)

# Directory with Medium_Turbidity images
medium_dir = os.path.join(path_med)

# Directory with High_Turbidity images
high_dir = os.path.join(path_high)

In [ ]:
vld=len(os.listdir(very_low_dir))
ld=len(os.listdir(low_dir))
md=len(os.listdir(medium_dir))
hd=len(os.listdir(high_dir))
print('total very_Low_Turbidity images:', vld)
print('total Low_Turbidity images:', ld)
print('total Medium_Turbidity images:', md)
print('total High_Turbidity images:', hd)

In [ ]:
from mahotas.labeled import label
import pandas as pd
arr_df=pd.DataFrame(columns=[['Angular Second Moment D1','Contrast D1','Correlation D1','Variance D1','Inverse Difference Moment D1','Sum Average D1','Sum Variance D1','Sum Entropy D1','Entropy D1','Difference Variance D1','Difference Entropy D1','Measure of correlation1 D1','Measure of correlation 2 D1',
                                         'Angular Second Moment D2','Contrast D2','Correlation D2','Variance D2','Inverse Difference Moment D2','Sum Average D2','Sum Variance D2','Sum Entropy D2','Entropy D2','Difference Variance D2','Difference Entropy D2','Measure of correlation1 D2','Measure of correlation 2 D2',
                                         'Angular Second Moment D3','Contrast D3','Correlation D3','Variance D3','Inverse Difference Moment D3','Sum Average D3','Sum Variance D3','Sum Entropy D3','Entropy D3','Difference Variance D3','Difference Entropy D3','Measure of correlation1 D3','Measure of correlation 2 D3',
                                         'Angular Second Moment D4','Contrast D4','Correlation D4','Variance D4','Inverse Difference Moment D4','Sum Average D4','Sum Variance D4','Sum Entropy D4','Entropy D4','Difference Variance D4','Difference Entropy D4','Measure of correlation1 D4','Measure of correlation 2 D4',
                                         'lbp similarity class1','lbp similarity class2','lbp similarity class3','lbp similarity class4','HOG Similarity class1','HOG Similarity class2','HOG Similarity class3','HOG Similarity class4','Mean Blue','Mean Green','Mean Red','Sd Blue','Sd Green','Sd Red','mean hue','mean saturation','mean light','sd hue','sd saturation','sd light','Similarity BGR Class 1','Similarity BGR Class 2','Similarity BGR Class 3','Similarity BGR Class 4','Similarity HSV Class 1','Similarity HSV Class 2','Similarity HSV Class 3','Similarity HSV Class 4','Blue proportion','Green Proportion','Red Proportion','Colourflness','Contrast','Brightness','Sharpness','Label']])

In [ ]:
#adding very low directory images
liste=[]
for filename in os.listdir(very_low_dir):
  input_path=os.path.join(very_low_dir,filename)
  d=haralick_extractor(input_path)
  ho1,ho2,ho3,ho4=hog_extractor(input_path)
  sc1,sc2,sc3,sc4=lbp_extractor(input_path)
  m1,m2,m3,sd1,sd2,sd3=mean_std_bgr(input_path)
  mh1,mh2,mh3,sh1,sh2,sh3=mean_std_hsv(input_path)
  s1,s2,s3,s4=hist_bgr(input_path)
  so1,so2,so3,so4=hist_hsv(input_path)
  b,g,r=bgr_distribution(input_path)
  colourflness=image_colorfulness(input_path)
  cont=contrast(input_path)
  bright=brightness(input_path)
  sharp=sharpness(input_path)

  liste= [*d,sc1,sc2,sc3,sc4,ho1,ho2,ho3,ho4,m1,m2,m3,sd1,sd2,sd3,mh1,mh2,mh3,sh1,sh2,sh3,s1,s2,s3,s4,so1,so2,so3,so4,b,g,r,colourflness,cont,bright,sharp,1] 

  arr=pd.Series(data=liste,index=[['Angular Second Moment D1','Contrast D1','Correlation D1','Variance D1','Inverse Difference Moment D1','Sum Average D1','Sum Variance D1','Sum Entropy D1','Entropy D1','Difference Variance D1','Difference Entropy D1','Measure of correlation1 D1','Measure of correlation 2 D1',
                                         'Angular Second Moment D2','Contrast D2','Correlation D2','Variance D2','Inverse Difference Moment D2','Sum Average D2','Sum Variance D2','Sum Entropy D2','Entropy D2','Difference Variance D2','Difference Entropy D2','Measure of correlation1 D2','Measure of correlation 2 D2',
                                         'Angular Second Moment D3','Contrast D3','Correlation D3','Variance D3','Inverse Difference Moment D3','Sum Average D3','Sum Variance D3','Sum Entropy D3','Entropy D3','Difference Variance D3','Difference Entropy D3','Measure of correlation1 D3','Measure of correlation 2 D3',
                                         'Angular Second Moment D4','Contrast D4','Correlation D4','Variance D4','Inverse Difference Moment D4','Sum Average D4','Sum Variance D4','Sum Entropy D4','Entropy D4','Difference Variance D4','Difference Entropy D4','Measure of correlation1 D4','Measure of correlation 2 D4',
                                         'lbp similarity class1','lbp similarity class2','lbp similarity class3','lbp similarity class4','HOG Similarity class1','HOG Similarity class2','HOG Similarity class3','HOG Similarity class4','Mean Blue','Mean Green','Mean Red','Sd Blue','Sd Green','Sd Red','mean hue','mean saturation','mean light','sd hue','sd saturation','sd light','Similarity BGR Class 1','Similarity BGR Class 2','Similarity BGR Class 3','Similarity BGR Class 4','Similarity HSV Class 1','Similarity HSV Class 2','Similarity HSV Class 3','Similarity HSV Class 4','Blue proportion','Green Proportion','Red Proportion','Colourflness','Contrast','Brightness','Sharpness','Label']])
  print(arr)
  arr_df=arr_df.append(arr,ignore_index=True)

In [ ]:
#adding low directory images
liste=[]
for filename in os.listdir(low_dir):
  input_path=os.path.join(low_dir,filename)
  d=haralick_extractor(input_path)
  ho1,ho2,ho3,ho4=hog_extractor(input_path)
  sc1,sc2,sc3,sc4=lbp_extractor(input_path)
  m1,m2,m3,sd1,sd2,sd3=mean_std_bgr(input_path)
  mh1,mh2,mh3,sh1,sh2,sh3=mean_std_hsv(input_path)
  s1,s2,s3,s4=hist_bgr(input_path)
  so1,so2,so3,so4=hist_hsv(input_path)
  b,g,r=bgr_distribution(input_path)
  colourflness=image_colorfulness(input_path)
  cont=contrast(input_path)
  bright=brightness(input_path)
  sharp=sharpness(input_path)

  liste= [*d,sc1,sc2,sc3,sc4,ho1,ho2,ho3,ho4,m1,m2,m3,sd1,sd2,sd3,mh1,mh2,mh3,sh1,sh2,sh3,s1,s2,s3,s4,so1,so2,so3,so4,b,g,r,colourflness,cont,bright,sharp,2] 

  arr=pd.Series(data=liste,index=[['Angular Second Moment D1','Contrast D1','Correlation D1','Variance D1','Inverse Difference Moment D1','Sum Average D1','Sum Variance D1','Sum Entropy D1','Entropy D1','Difference Variance D1','Difference Entropy D1','Measure of correlation1 D1','Measure of correlation 2 D1',
                                         'Angular Second Moment D2','Contrast D2','Correlation D2','Variance D2','Inverse Difference Moment D2','Sum Average D2','Sum Variance D2','Sum Entropy D2','Entropy D2','Difference Variance D2','Difference Entropy D2','Measure of correlation1 D2','Measure of correlation 2 D2',
                                         'Angular Second Moment D3','Contrast D3','Correlation D3','Variance D3','Inverse Difference Moment D3','Sum Average D3','Sum Variance D3','Sum Entropy D3','Entropy D3','Difference Variance D3','Difference Entropy D3','Measure of correlation1 D3','Measure of correlation 2 D3',
                                         'Angular Second Moment D4','Contrast D4','Correlation D4','Variance D4','Inverse Difference Moment D4','Sum Average D4','Sum Variance D4','Sum Entropy D4','Entropy D4','Difference Variance D4','Difference Entropy D4','Measure of correlation1 D4','Measure of correlation 2 D4',
                                         'lbp similarity class1','lbp similarity class2','lbp similarity class3','lbp similarity class4','HOG Similarity class1','HOG Similarity class2','HOG Similarity class3','HOG Similarity class4','Mean Blue','Mean Green','Mean Red','Sd Blue','Sd Green','Sd Red','mean hue','mean saturation','mean light','sd hue','sd saturation','sd light','Similarity BGR Class 1','Similarity BGR Class 2','Similarity BGR Class 3','Similarity BGR Class 4','Similarity HSV Class 1','Similarity HSV Class 2','Similarity HSV Class 3','Similarity HSV Class 4','Blue proportion','Green Proportion','Red Proportion','Colourflness','Contrast','Brightness','Sharpness','Label']])
  print(arr)
  arr_df=arr_df.append(arr,ignore_index=True)

In [ ]:
#adding high directory images
liste=[]
for filename in os.listdir(high_dir):
  input_path=os.path.join(high_dir,filename)
  d=haralick_extractor(input_path)
  ho1,ho2,ho3,ho4=hog_extractor(input_path)
  sc1,sc2,sc3,sc4=lbp_extractor(input_path)
  m1,m2,m3,sd1,sd2,sd3=mean_std_bgr(input_path)
  mh1,mh2,mh3,sh1,sh2,sh3=mean_std_hsv(input_path)
  s1,s2,s3,s4=hist_bgr(input_path)
  so1,so2,so3,so4=hist_hsv(input_path)
  b,g,r=bgr_distribution(input_path)
  colourflness=image_colorfulness(input_path)
  cont=contrast(input_path)
  bright=brightness(input_path)
  sharp=sharpness(input_path)

  liste= [*d,sc1,sc2,sc3,sc4,ho1,ho2,ho3,ho4,m1,m2,m3,sd1,sd2,sd3,mh1,mh2,mh3,sh1,sh2,sh3,s1,s2,s3,s4,so1,so2,so3,so4,b,g,r,colourflness,cont,bright,sharp,4] 

  arr=pd.Series(data=liste,index=[['Angular Second Moment D1','Contrast D1','Correlation D1','Variance D1','Inverse Difference Moment D1','Sum Average D1','Sum Variance D1','Sum Entropy D1','Entropy D1','Difference Variance D1','Difference Entropy D1','Measure of correlation1 D1','Measure of correlation 2 D1',
                                         'Angular Second Moment D2','Contrast D2','Correlation D2','Variance D2','Inverse Difference Moment D2','Sum Average D2','Sum Variance D2','Sum Entropy D2','Entropy D2','Difference Variance D2','Difference Entropy D2','Measure of correlation1 D2','Measure of correlation 2 D2',
                                         'Angular Second Moment D3','Contrast D3','Correlation D3','Variance D3','Inverse Difference Moment D3','Sum Average D3','Sum Variance D3','Sum Entropy D3','Entropy D3','Difference Variance D3','Difference Entropy D3','Measure of correlation1 D3','Measure of correlation 2 D3',
                                         'Angular Second Moment D4','Contrast D4','Correlation D4','Variance D4','Inverse Difference Moment D4','Sum Average D4','Sum Variance D4','Sum Entropy D4','Entropy D4','Difference Variance D4','Difference Entropy D4','Measure of correlation1 D4','Measure of correlation 2 D4',
                                         'lbp similarity class1','lbp similarity class2','lbp similarity class3','lbp similarity class4','HOG Similarity class1','HOG Similarity class2','HOG Similarity class3','HOG Similarity class4','Mean Blue','Mean Green','Mean Red','Sd Blue','Sd Green','Sd Red','mean hue','mean saturation','mean light','sd hue','sd saturation','sd light','Similarity BGR Class 1','Similarity BGR Class 2','Similarity BGR Class 3','Similarity BGR Class 4','Similarity HSV Class 1','Similarity HSV Class 2','Similarity HSV Class 3','Similarity HSV Class 4','Blue proportion','Green Proportion','Red Proportion','Colourflness','Contrast','Brightness','Sharpness','Label']])
  print(arr)
  arr_df=arr_df.append(arr,ignore_index=True)

In [ ]:
#adding medium directory images
liste=[]
for filename in os.listdir(medium_dir):
  input_path=os.path.join(medium_dir,filename)
  d=haralick_extractor(input_path)
  ho1,ho2,ho3,ho4=hog_extractor(input_path)
  sc1,sc2,sc3,sc4=lbp_extractor(input_path)
  m1,m2,m3,sd1,sd2,sd3=mean_std_bgr(input_path)
  mh1,mh2,mh3,sh1,sh2,sh3=mean_std_hsv(input_path)
  s1,s2,s3,s4=hist_bgr(input_path)
  so1,so2,so3,so4=hist_hsv(input_path)
  b,g,r=bgr_distribution(input_path)
  colourflness=image_colorfulness(input_path)
  cont=contrast(input_path)
  bright=brightness(input_path)
  sharp=sharpness(input_path)

  liste= [*d,sc1,sc2,sc3,sc4,ho1,ho2,ho3,ho4,m1,m2,m3,sd1,sd2,sd3,mh1,mh2,mh3,sh1,sh2,sh3,s1,s2,s3,s4,so1,so2,so3,so4,b,g,r,colourflness,cont,bright,sharp,3] 

  arr=pd.Series(data=liste,index=[['Angular Second Moment D1','Contrast D1','Correlation D1','Variance D1','Inverse Difference Moment D1','Sum Average D1','Sum Variance D1','Sum Entropy D1','Entropy D1','Difference Variance D1','Difference Entropy D1','Measure of correlation1 D1','Measure of correlation 2 D1',
                                         'Angular Second Moment D2','Contrast D2','Correlation D2','Variance D2','Inverse Difference Moment D2','Sum Average D2','Sum Variance D2','Sum Entropy D2','Entropy D2','Difference Variance D2','Difference Entropy D2','Measure of correlation1 D2','Measure of correlation 2 D2',
                                         'Angular Second Moment D3','Contrast D3','Correlation D3','Variance D3','Inverse Difference Moment D3','Sum Average D3','Sum Variance D3','Sum Entropy D3','Entropy D3','Difference Variance D3','Difference Entropy D3','Measure of correlation1 D3','Measure of correlation 2 D3',
                                         'Angular Second Moment D4','Contrast D4','Correlation D4','Variance D4','Inverse Difference Moment D4','Sum Average D4','Sum Variance D4','Sum Entropy D4','Entropy D4','Difference Variance D4','Difference Entropy D4','Measure of correlation1 D4','Measure of correlation 2 D4',
                                         'lbp similarity class1','lbp similarity class2','lbp similarity class3','lbp similarity class4','HOG Similarity class1','HOG Similarity class2','HOG Similarity class3','HOG Similarity class4','Mean Blue','Mean Green','Mean Red','Sd Blue','Sd Green','Sd Red','mean hue','mean saturation','mean light','sd hue','sd saturation','sd light','Similarity BGR Class 1','Similarity BGR Class 2','Similarity BGR Class 3','Similarity BGR Class 4','Similarity HSV Class 1','Similarity HSV Class 2','Similarity HSV Class 3','Similarity HSV Class 4','Blue proportion','Green Proportion','Red Proportion','Colourflness','Contrast','Brightness','Sharpness','Label']])
  print(arr)
  arr_df=arr_df.append(arr,ignore_index=True)

Data Cleaning

In [ ]:
import sklearn
arr_df = sklearn.utils.shuffle(arr_df)

In [ ]:
arr_df

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(arr_df.iloc[:,:-1], arr_df.iloc[:,-1:], test_size=0.33, random_state=42)

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=16, random_state=42,n_estimators=50)
clf.fit(X_train, y_train)



In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(clf, X_test, y_test)  
plt.show()

In [ ]:
cm=confusion_matrix(y_test,y_pred)

In [ ]:
#SVM
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
clf2 = SVC(gamma='auto')
clf2.fit(X_train, y_train)

In [ ]:
y_pred2=clf2.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred2)

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(clf2, X_test, y_test)  
plt.show()

In [ ]:
# XGBOOST
from xgboost import XGBClassifier
model = XGBClassifier(max_depth=16,n_estimators=50)
model.fit(X_train, y_train)

# make predictions for test data

y_pred3 = model.predict(X_test)
predictions = [round(value) for value in y_pred3]

# evaluate predictions

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(clf, X_test, y_test)  
plt.show()